In [ ]:
import pandas as pd  # таблицы
import os

import spacy
nlp = spacy.load("ru_core_news_lg")

import plotly.graph_objects as go
from plotly.subplots import make_subplots

dir = "D:\Develop\VS\HSE\Pump_Dump\\"  # папка проекта

In [ ]:
# загружаем данные общего индекса активности IMOEX

imoex_path = dir + "Market\\MOEX Russia Historical Data.csv"
imoex_df = pd.read_csv(
    imoex_path,
    encoding="utf-8",
    keep_default_na=False,
)
# удаляем лишние столбцы и символы в названиях столбцов
imoex_df.rename(columns={"Change %":"Change"},inplace=True)
imoex_df["Change"] = imoex_df["Change"].str.replace('%', '')
imoex_df.drop('Vol.', axis= 1 , inplace= True )
imoex_df["Date"] = imoex_df["Date"].astype("datetime64[ns]")

# формируем корректные для обработки значения для числовых столбцов
cols_float=["Price", "Low", "Open", "High", "Change"]

imoex_df[cols_float] = imoex_df[cols_float].apply(lambda x: x.str.replace(",", ""))
imoex_df[cols_float] = imoex_df[cols_float].apply(pd.to_numeric)

# рассчитываем дату закрытия
imoex_df["Close"] = (imoex_df['Open']+(imoex_df['Open']/100*imoex_df['Change'])).round(2)

imoex_df.sort_values(by="Date")

# добавляем значения цены до и после
imoex_df["Before"] = imoex_df["Price"].shift(-1)
imoex_df["After"] = imoex_df["Price"].shift(1)
# удаляем пустые, т.к. для первой и последней строки будут отсуствовать значения
imoex_df.dropna(inplace=True)

imoex_df.to_csv(dir + "Answers\\imoex_changes.csv", encoding="utf-8", sep="|")

In [ ]:
# загружаем котировки по всем акциям
files = [_ for _ in os.listdir(dir + "Market\\Trades\\") if _.endswith(r".csv")]
df_temp = []
for file in files:
    df_temp.append(pd.read_csv(dir + "Market\\Trades\\" + file, encoding="utf-8", sep="|"))
tickers_dfs = pd.concat([df for df in df_temp if not df.empty], ignore_index=True)

def quotes_stat(tickers_list, df, roll_period): # расчет скользящего среднего и отклонения от него по списку тикеров
    pd.options.mode.chained_assignment = None
    tic_quot_df = pd.DataFrame()
    for ticker in tickers_list:
        temp_df = df.loc[df["SECID"]==ticker]
        temp_df["ROLL"] = temp_df.groupby(["SECID"])["WAPRICE"].transform(lambda x: x.rolling (roll_period, 1).mean()).round(3)
        temp_df["DEV"]  = round((temp_df["WAPRICE"]-temp_df["ROLL"])/temp_df["WAPRICE"],3)
        tic_quot_df = pd.concat([tic_quot_df, temp_df])
    return tic_quot_df

# удаляем ненужные столбцы, приводим необходимые типы данных
tickers_dfs.drop(["NUMTRADES", "VALUE"], axis= 1 , inplace= True )
tickers_dfs["SECID"] = tickers_dfs["SECID"].str.lower()
tickers_dfs["TRADEDATE"] = tickers_dfs["TRADEDATE"].astype("datetime64[ns]")
tickers_dfs["WAPRICE"] = tickers_dfs["WAPRICE"].apply(pd.to_numeric)
# обязательно сортировать по возрастаннию для корректных расчетов скользящего
tickers_dfs.sort_values(by="TRADEDATE", ascending=True, inplace=True)

# берем все уникальные тикеры из фрейма
tickers_list = pd.unique(tickers_dfs["SECID"])
limit = 0.05 # отклонение цены от скользящего среднего (%/100)
roll_period = 5 # количество дней для расчета скользящего среднего
# формируем базу
stat_df = quotes_stat(tickers_list, tickers_dfs, roll_period)
stat_df.dropna(inplace=True)
# сохраняем на диск
# stat_df.to_csv(dir + "Answers\\alltickers.csv", encoding="utf-8", sep="|")

In [ ]:
# загружаем все новости по списку каналов
channel_path = dir + "TG\\" + "!channels" + ".csv"
channels_df = pd.read_csv(
    channel_path,
    encoding="utf-8",
    sep="|",
    keep_default_na=False,
)
big_df = pd.DataFrame()
for channel in channels_df["CHANNEL"].values:
    channel_path = dir + "TG\\" + channel + "_post.csv"
    # загрузка файла
    temp_df = pd.read_csv(
        channel_path,
        encoding="utf-8",
        sep="|",
        keep_default_na=False,
        usecols=["NEWS_DATE", "CHANNEL", "NEWS_TEXT", "TICKERS", "WORDS"],
    )
    big_df = pd.concat([big_df, temp_df], ignore_index=False)
# проводим необходимые преобразования в таблице
big_df['NEWS_DATE'] = pd.to_datetime(big_df["NEWS_DATE"], format='%Y-%m-%d')
big_df['WORDS'] = big_df['WORDS'].str.replace(","," ")
big_df.drop_duplicates(subset=['WORDS'], keep='first', inplace=True)
big_df.sort_values(by=["NEWS_DATE", "CHANNEL"], inplace=True, ascending=False)
print(big_df["CHANNEL"].value_counts())
big_df.head()

In [ ]:
# лемматизируем токены и записываем в отдельное поле
def preprocess_and_lemmatize(text):
    # text = " ".join(text)
    doc = nlp(text)  # Привести к нижнему регистру и лемматизировать
    lemmatized_words = [token.lemma_ for token in doc]
    lemmatized_words = " ".join(lemmatized_words)
    return lemmatized_words

big_df["WORDS_lemma"] = big_df["WORDS"].apply(preprocess_and_lemmatize)
big_df.head()

In [ ]:
# добавляем к значениям с ТИКЕРОМ цену на дату новости
main_df = pd.merge(
    big_df, tickers_dfs, left_on=["NEWS_DATE", "TICKERS"], right_on=["TRADEDATE", "SECID"], how="left"
)
main_df.drop(["SECID"], inplace=True, axis=1)
main_df.drop(["TRADEDATE"], inplace=True, axis=1)

In [ ]:
# сохраняем собранный фрейм на диск
# main_df.to_csv(dir + "Answers\\news_quotes.csv", encoding="utf-8", sep="|")

In [ ]:
# создаем график с динамикой индекса и количеством новостей

# группируем новости по дате
groupNews = big_df.groupby("NEWS_DATE").size().to_frame(name = 'count').reset_index()

fig = make_subplots(specs=[[{"secondary_y": True}]])
# задаем отображение количества новостей
fig.add_trace(
    go.Bar(
        name = "Новости",
        x=groupNews["NEWS_DATE"], y=groupNews["count"],
        marker=dict(color="lightsteelblue"),
        opacity=0.5,
    ),
    secondary_y=True
)
# задаем отображение динамики индекса
fig.add_trace(
    go.Candlestick(
        name = "Цена",
        x=imoex_df['Date'],
        open=imoex_df['Open'],
        high=imoex_df['High'],
        low=imoex_df['Low'],
        close=imoex_df['Open']+(imoex_df['Open']/100*imoex_df['Change']),
    ),
    secondary_y=False
)
# добавляем заголовок
fig.update_layout(title = 'Динамика <b>IMOEX</b>', xaxis_rangeslider_visible=False)

fig.show()
